In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Feature Analysis for Patient Readmission Prediction\n",
    "\n",
    "This notebook performs comprehensive feature analysis including:\n",
    "- Feature importance analysis\n",
    "- Feature selection techniques\n",
    "- Statistical tests\n",
    "- Feature engineering evaluation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import plotly.express as px\n",
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "\n",
    "# Machine Learning libraries\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier\n",
    "from sklearn.feature_selection import (\n",
    "    SelectKBest, chi2, f_classif, mutual_info_classif,\n",
    "    RFE, SelectFromModel, VarianceThreshold\n",
    ")\n",
    "from sklearn.preprocessing import StandardScaler, LabelEncoder\n",
    "from sklearn.linear_model import LogisticRegression\n",
    "from sklearn.metrics import classification_report, roc_auc_score\n",
    "\n",
    "# Statistical tests\n",
    "from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu\n",
    "from scipy import stats\n",
    "\n",
    "# Database\n",
    "from sqlalchemy import create_engine\n",
    "\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set styling\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "pd.set_option('display.max_columns', None)\n",
    "\n",
    "print(\"Libraries imported successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Data Loading and Preprocessing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load processed data\n",
    "try:\n",
    "    # Try to load from feature engineering output\n",
    "    df = pd.read_csv('../data/processed/engineered_features.csv')\n",
    "    print(\"Loaded engineered features from CSV\")\n",
    "except FileNotFoundError:\n",
    "    try:\n",
    "        # Fallback to cleaned data\n",
    "        df = pd.read_csv('../data/processed/cleaned_data.csv')\n",
    "        print(\"Loaded cleaned data from CSV\")\n",
    "    except FileNotFoundError:\n",
    "        # Fallback to database\n",
    "        engine = create_engine('sqlite:///../database/patient_readmission.db')\n",
    "        query = \"\"\"\n",
    "        SELECT \n",
    "            p.patient_id,\n",
    "            p.age,\n",
    "            p.gender,\n",
    "            p.race,\n",
    "            p.admission_type_id,\n",
    "            p.discharge_disposition_id,\n",
    "            p.admission_source_id,\n",
    "            p.time_in_hospital as length_of_stay,\n",
    "            p.num_lab_procedures,\n",
    "            p.num_procedures,\n",
    "            p.num_medications,\n",
    "            p.number_outpatient,\n",
    "            p.number_emergency,\n",
    "            p.number_inpatient,\n",
    "            p.number_diagnoses as num_diagnoses,\n",
    "            p.max_glu_serum,\n",
    "            p.A1Cresult,\n",
    "            p.change,\n",
    "            p.diabetesMed,\n",
    "            p.readmitted,\n",
    "            CASE WHEN p.readmitted IN ('<30', 'YES') THEN 1 ELSE 0 END as readmitted_30_days\n",
    "        FROM patients p\n",
    "        WHERE p.age != '[0-10)'\n",
    "        \"\"\"\n",
    "        df = pd.read_sql_query(query, engine)\n",
    "        print(\"Loaded raw data from database\")\n",
    "\n",
    "print(f\"Dataset shape: {df.shape}\")\n",
    "print(f\"Columns: {list(df.columns)}\")\n",
    "print(f\"Target variable exists: {'readmitted_30_days' in df.columns}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Basic preprocessing\n",
    "print(\"=== DATA PREPROCESSING ===\")\n",
    "\n",
    "# Create target if doesn't exist\n",
    "if 'readmitted_30_days' not in df.columns and 'readmitted' in df.columns:\n",
    "    df['readmitted_30_days'] = (df['readmitted'] == '<30').astype(int)\n",
    "    print(\"Created readmitted_30_days target variable\")\n",
    "\n",
    "# Remove patient_id for analysis (keep for tracking if needed)\n",
    "analysis_df = df.drop(columns=['patient_id'], errors='ignore')\n",
    "\n",
    "# Handle target variable\n",
    "if 'readmitted_30_days' in analysis_df.columns:\n",
    "    target = 'readmitted_30_days'\n",
    "    y = analysis_df[target]\n",
    "    X = analysis_df.drop(columns=[target, 'readmitted'], errors='ignore')\n",
    "    print(f\"Features shape: {X.shape}\")\n",
    "    print(f\"Target distribution: {y.value_counts().to_dict()}\")\n",
    "    print(f\"Class balance: {y.value_counts(normalize=True).round(3).to_dict()}\")\n",
    "else:\n",
    "    print(\"Warning: Target variable not found. Creating dummy target for analysis.\")\n",
    "    y = np.random.choice([0, 1], size=len(analysis_df), p=[0.85, 0.15])\n",
    "    X = analysis_df\n",
    "\n",
    "print(f\"\\nMissing values per column:\")\n",
    "missing_info = X.isnull().sum()\n",
    "print(missing_info[missing_info > 0].sort_values(ascending=False))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Univariate Feature Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Separate numerical and categorical features\n",
    "numerical_features = X.select_dtypes(include=[np.number]).columns.tolist()\n",
    "categorical_features = X.select_dtypes(include=['object']).columns.tolist()\n",
    "\n",
    "print(f\"Numerical features ({len(numerical_features)}): {numerical_features}\")\n",
    "print(f\"Categorical features ({len(categorical_features)}): {categorical_features}\")\n",
    "\n",
    "# Remove constant features\n",
    "constant_features = []\n",
    "for col in X.columns:\n",
    "    if X[col].nunique() <= 1:\n",
    "        constant_features.append(col)\n",
    "\n",
    "if constant_features:\n",
    "    print(f\"\\nConstant features to remove: {constant_features}\")\n",
    "    X = X.drop(columns=constant_features)\n",
    "    numerical_features = [col for col in numerical_features if col not in constant_features]\n",
    "    categorical_features = [col for col in categorical_features if col not in constant_features]\n",
    "\n",
    "print(f\"\\nFinal feature counts - Numerical: {len(numerical_features)}, Categorical: {len(categorical_features)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Statistical tests for numerical features\n",
    "print(\"=== STATISTICAL TESTS FOR NUMERICAL FEATURES ===\")\n",
    "numerical_stats = []\n",
    "\n",
    "for feature in numerical_features:\n",
    "    if feature in X.columns:\n",
    "        # Remove missing values for analysis\n",
    "        feature_data = X[feature].dropna()\n",
    "        target_data = y[feature_data.index]\n",
    "        \n",
    "        # Separate by target classes\n",
    "        group_0 = feature_data[target_data == 0]\n",
    "        group_1 = feature_data[target_data == 1]\n",
    "        \n",
    "        if len(group_0) > 0 and len(group_1) > 0:\n",
    "            # T-test\n",
    "            try:\n",
    "                t_stat, t_pvalue = ttest_ind(group_0, group_1)\n",
    "            except:\n",
    "                t_stat, t_pvalue = np.nan, np.nan\n",
    "            \n",
    "            # Mann-Whitney U test (non-parametric)\n",
    "            try:\n",
    "                u_stat, u_pvalue = mannwhitneyu(group_0, group_1, alternative='two-sided')\n",
    "            except:\n",
    "                u_stat, u_pvalue = np.nan, np.nan\n",
    "            \n",
    "            numerical_stats.append({\n",
    "                'Feature': feature,\n",
    "                'Mean_No_Readmit': group_0.mean(),\n",
    "                'Mean_Readmit': group_1.mean(),\n",
    "                'Std_No_Readmit': group_0.std(),\n",
    "                'Std_Readmit': group_1.std(),\n",
    "                'T_Statistic': t_stat,\n",
    "                'T_P_Value': t_pvalue,\n",
    "                'U_P_Value': u_pvalue,\n",
    "                'Effect_Size': abs(group_1.mean() - group_0.mean()) / np.sqrt((group_0.var() + group_1.var()) / 2)\n",
    "            })\n",
    "\n",
    "numerical_stats_df = pd.DataFrame(numerical_stats)\n",
    "if len(numerical_stats_df) > 0:\n",
    "    numerical_stats_df = numerical_stats_df.sort_values('T_P_Value')\n",
    "    print(\"Top numerical features by statistical significance:\")\n",
    "    print(numerical_stats_df.round(4).head(10))\n",
    "    \n",
    "    # Visualize top significant features\n",
    "    significant_features = numerical_stats_df[numerical_stats_df['T_P_Value'] < 0.05].head(6)\n",
    "    if len(significant_features) > 0:\n",
    "        fig, axes = plt.subplots(2, 3, figsize=(15, 10))\n",
    "        axes = axes.ravel()\n",
    "        \n",
    "        for i, (_, row) in enumerate(significant_features.iterrows()):\n",
    "            feature = row['Feature']\n",
    "            feature_data = X[feature].dropna()\n",
    "            target_data = y[feature_data.index]\n",
    "            \n",
    "            # Box plot\n",
    "            data_to_plot = [feature_data[target_data == 0], feature_data[target_data == 1]]\n",
    "            axes[i].boxplot(data_to_plot, labels=['No Readmit', 'Readmit'])\n",
    "            axes[i].set_title(f'{feature}\\n(p-value: {row[\"T_P_Value\"]:.4f})')\n",
    "            axes[i].set_ylabel('Value')\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Statistical tests for categorical features\n",
    "print(\"\\n=== STATISTICAL TESTS FOR CATEGORICAL FEATURES ===\")\n",
    "categorical_stats = []\n",
    "\n",
    "for feature in categorical_features:\n",
    "    if feature in X.columns:\n",
    "        # Create contingency table\n",
    "        try:\n",
    "            # Handle missing values\n",
    "            feature_data = X[feature].fillna('Missing')\n",
    "            contingency_table = pd.crosstab(feature_data, y)\n",
    "            \n",
    "            # Chi-square test\n",
    "            chi2_stat, chi2_pvalue, dof, expected = chi2_contingency(contingency_table)\n",
    "            \n",
    "            # Cramér's V (effect size)\n",
    "            n = contingency_table.sum().sum()\n",
    "            cramers_v = np.sqrt(chi2_stat / (n * (min(contingency_table.shape) - 1)))\n",
    "            \n",
    "            categorical_stats.append({\n",
    "                'Feature': feature,\n",
    "                'Unique_Values': X[feature].nunique(),\n",
    "                'Chi2_Statistic': chi2_stat,\n",
    "                'Chi2_P_Value': chi2_pvalue,\n",
    "                'Cramers_V': cramers_v,\n",
    "                'DOF': dof\n",
    "            })\n",
    "        except Exception as e:\n",
    "            print(f\"Error processing {feature}: {e}\")\n",
    "\n",
    "categorical_stats_df = pd.DataFrame(categorical_stats)\n",
    "if len(categorical_stats_df) > 0:\n",
    "    categorical_stats_df = categorical_stats_df.sort_values('Chi2_P_Value')\n",
    "    print(\"Top categorical features by statistical significance:\")\n",
    "    print(categorical_stats_df.round(4).head(10))\n",
    "    \n",
    "    # Visualize top significant categorical features\n",
    "    significant_cat = categorical_stats_df[categorical_stats_df['Chi2_P_Value'] < 0.05].head(4)\n",
    "    if len(significant_cat) > 0:\n",
    "        fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "        axes = axes.ravel()\n",
    "        \n",
    "        for i, (_, row) in enumerate(significant_cat.iterrows()):\n",
    "            feature = row['Feature']\n",
    "            crosstab = pd.crosstab(X[feature].fillna('Missing'), y, normalize='columns')\n",
    "            crosstab.plot(kind='bar', ax=axes[i], rot=45)\n",
    "            axes[i].set_title(f'{feature}\\n(p-value: {row[\"Chi2_P_Value\"]:.4f})')\n",
    "            axes[i].set_ylabel('Proportion')\n",
    "            axes[i].legend(['No Readmit', 'Readmit'])\n",
    "        \n",
    "        plt.tight_layout()\n",
    "        plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Feature Importance Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Prepare data for machine learning\n",
    "print(\"=== PREPARING DATA FOR ML ===\")\n",
    "X_processed = X.copy()\n",
    "\n",
    "# Fill missing values\n",
    "for col in numerical_features:\n",
    "    if col in X_processed.columns:\n",
    "        X_processed[col] = X_processed[col].fillna(X_processed[col].median())\n",
    "\n",
    "# Encode categorical variables\n",
    "label_encoders = {}\n",
    "for col in categorical_features:\n",
    "    if col in X_processed.columns:\n",
    "        le = LabelEncoder()\n",
    "        X_processed[col] = X_processed[col].fillna('Unknown')\n",
    "        X_processed[col] = le.fit_transform(X_processed[col].astype(str))\n",
    "        label_encoders[col] = le\n",
    "\n",
    "print(f\"Processed dataset shape: {X_processed.shape}\")\n",
    "print(f\"Missing values remaining: {X_processed.isnull().sum().sum()}\")\n",
    "print(f\"Features: {list(X_processed.columns)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Random Forest Feature Importance\n",
    "print(\"=== RANDOM FOREST FEATURE IMPORTANCE ===\")\n",
    "rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)\n",
    "rf.fit(X_processed, y)\n",
    "\n",
    "# Get feature importance\n",
    "rf_importance = pd.DataFrame({\n",
    "    'Feature': X_processed.columns,\n",
    "    'Importance': rf.feature_importances_\n",
    "}).sort_values('Importance', ascending=False)\n",
    "\n",
    "print(\"Top 15 features by Random Forest importance:\")\n",
    "print(rf_importance.head(15))\n",
    "\n",
    "# Visualize top features\n",
    "plt.figure(figsize=(12, 8))\n",
    "top_features = rf_importance.head(15)\n",
    "sns.barplot(data=top_features, x='Importance', y='Feature')\n",
    "plt.title('Top 15 Features - Random Forest Importance')\n",
    "plt.xlabel('Feature Importance')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Feature importance distribution\n",
    "plt.figure(figsize=(10, 6))\n",
    "plt.hist(rf_importance['Importance'], bins=20, alpha=0.7, edgecolor='black')\n",
    "plt.title('Distribution of Feature Importance Scores')\n",
    "plt.xlabel('Importance Score')\n",
    "plt.ylabel('Number of Features')\n",
    "plt.axvline(rf_importance['Importance'].mean(), color='red', linestyle='--', label='Mean')\n",
    "plt.legend()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Extra Trees Feature Importance\n",
    "print(\"\\n=== EXTRA TREES FEATURE IMPORTANCE ===\")\n",
    "et = ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1)\n",
    "et.fit(X_processed, y)\n",
    "\n",
    "et_importance = pd.DataFrame({\n",
    "    'Feature': X_processed.columns,\n",
    "    'Importance': et.feature_importances_\n",
    "}).sort_values('Importance', ascending=False)\n",
    "\n",
    "print(\"Top 15 features by Extra Trees importance:\")\n",
    "print(et_importance.head(15))\n",
    "\n",
    "# Compare Random Forest vs Extra Trees importance\n",
    "importance_comparison = rf_importance.merge(et_importance, on='Feature', suffixes=('_RF', '_ET'))\n",
    "importance_comparison['Avg_Importance'] = (importance_comparison['Importance_RF'] + importance_comparison['Importance_ET']) / 2\n",
    "importance_comparison = importance_comparison.sort_values('Avg_Importance', ascending=False)\n",
    "\n",
    "plt.figure(figsize=(12, 8))\n",
    "top_combined = importance_comparison.head(15)\n",
    "x = np.arange(len(top_combined))\n",
    "width = 0.35\n",
    "\n",
    "plt.bar(x - width/2, top_combined['Importance_RF'], width, label='Random Forest', alpha=0.8)\n",
    "plt.bar(x + width/2, top_combined['Importance_ET'], width, label='Extra Trees', alpha=0.8)\n",
    "\n",
    "plt.xlabel('Features')\n",
    "plt.ylabel('Importance')\n",
    "plt.title('Feature Importance Comparison: Random Forest vs Extra Trees')\n",
    "plt.xticks(x, top_combined['Feature'], rotation=45, ha='right')\n",
    "plt.legend()\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Correlation between RF and ET importance\n",
    "plt.figure(figsize=(8, 6))\n",
    "plt.scatter(importance_comparison['Importance_RF'], importance_comparison['Importance_ET'], alpha=0.6)\n",
    "plt.plot([0, importance_comparison['Importance_RF'].max()], [0, importance_comparison['Importance_RF'].max()], 'r--', alpha=0.8)\n",
    "plt.xlabel('Random Forest Importance')\n",
    "plt.ylabel('Extra Trees Importance')\n",
    "plt.title('Correlation between RF and ET Feature Importance')\n",
    "correlation = importance_comparison['Importance_RF'].corr(importance_comparison['Importance_ET'])\n",
    "plt.text(0.05, 0.95, f'Correlation: {correlation:.3f}', transform=plt.gca().transAxes, \n",
    "         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Feature Selection Techniques"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Univariate Feature Selection\n",
    "print(\"=== UNIVARIATE FEATURE SELECTION ===\")\n",
    "\n",
    "# For numerical features - F-score\n",
    "if len(numerical_features) > 0:\n",
    "    numerical_data = X_processed[[col for col in numerical_features if col in X_processed.columns]]\n",
    "    if len(numerical_data.columns) > 0:\n",
    "        f_scores, f_pvalues = f_classif(numerical_data, y)\n",
    "        \n",
    "        f_score_results = pd.DataFrame({\n",
    "            'Feature': numerical_data.columns,\n",
    "            'F_Score': f_scores,\n",
    "            'P_Value': f_pvalues\n",
    "        }).sort_values('F_Score', ascending=False)\n",
    "        \n",
    "        print(\"\\nTop numerical features by F-score:\")\n",
    "        print(f_score_results.head(10))\n",
    "\n",
    "# For all features - Mutual Information\n",
    "mi_scores = mutual_info_classif(X_processed, y, random_state=42)\n",
    "mi_results = pd.DataFrame({\n",
    "    'Feature': X_processed.columns,\n",
    "    'MI_Score': mi_scores\n",
    "}).sort_values('MI_Score', ascending=False)\n",
    "\n",
    "print(\"\\nTop features by Mutual Information:\")\n",
    "print(mi_results.head(15))\n",
    "\n",
    "# Visualize MI scores\n",
    "plt.figure(figsize=(12, 8))\n",
    "top_mi = mi_results.head(15)\n",
    "sns.barplot(data=top_mi, x='MI_Score', y='Feature')\n",
    "plt.title('Top 15 Features - Mutual Information Score')\n",
    "plt.xlabel('Mutual Information Score')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Recursive Feature Elimination (RFE)\n",
    "print(\"\\n=== RECURSIVE FEATURE ELIMINATION ===\")\n",
    "\n",
    "# Use Logistic Regression as base estimator\n",
    "lr = LogisticRegression(random_state=42, max_iter=1000)\n",
    "n_features_to_select = min(15, X_processed.shape[1])\n",
    "rfe = RFE(estimator=lr, n_features_to_select=n_features_to_select, step=1)\n",
    "\n",
    "# Scale features for logistic regression\n",
    "scaler = StandardScaler()\n",
    "X_scaled = scaler.fit_transform(X_processed)\n",
    "\n",
    "rfe.fit(X_scaled, y)\n",
    "\n",
    "rfe_results = pd.DataFrame({\n",
    "    'Feature': X_processed.columns,\n",
    "    'Selected': rfe.support_,\n",
    "    'Ranking': rfe.ranking_\n",
    "}).sort_values('Ranking')\n",
    "\n",
    "print(\"RFE Selected Features:\")\n",
    "selected_features = rfe_results[rfe_results['Selected']]['Feature'].tolist()\n",
    "for i, feature in enumerate(selected_features, 1):\n",
    "    print(f\"{i}. {feature}\")\n",
    "\n",
    "print(\"\\nRFE Feature Rankings (top 15):\")\n",
    "print(rfe_results.head(15))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# L1-based Feature Selection (Lasso)\n",
    "print(\"\\n=== L1-BASED FEATURE SELECTION (LASSO) ===\")\n",
    "\n",
    "# Use different C values to see feature selection\n",
    "C_values = [0.01, 0.1, 1.0, 10.0]\n",
    "lasso_results = {}\n",
    "\n",
    "for C in C_values:\n",
    "    lasso = LogisticRegression(penalty='l1', solver='liblinear', C=C, random_state=42)\n",
    "    selector = SelectFromModel(lasso)\n",
    "    selector.fit(X_scaled, y)\n",
    "    \n",
    "    selected_features = X_processed.columns[selector.get_support()].tolist()\n",
    "    lasso_results[C] = {\n",
    "        'n_features': len(selected_features),\n",
    "        'features': selected_features\n",
    "    }\n",
    "    \n",
    "    print(f\"C={C}: {len(selected_features)} features selected\")\n",
    "\n",
    "# Show features selected by most restrictive model\n",
    "if lasso_results[0.01]['features']:\n",
    "    print(f\"\\nFeatures selected by Lasso (C=0.01): {lasso_results[0.01]['features']}\")\n",
    "else:\n",
    "    print(f\"\\nFeatures selected by Lasso (C=0.1): {lasso_results[0.1]['features']}\")\n",
    "\n",
    "# Visualize feature selection across different C values\n",
    "plt.figure(figsize=(